In [79]:
import numpy as np
import matplotlib.pyplot as plt
import math
from numpy import random as rd

Functions that generate matrices :

In [80]:
def N(x): 
    #density function of Normal(mu, sigma) distribution
    temp=(x-mu)/sigma
    factor= -(temp*temp)/2
    quotient = sigma*math.sqrt(2*math.pi)
    y = math.exp(factor) /quotient
    return y

In [81]:
def g_mat(n): 
    #generate an nxn matrix with normally ditributed entries
    mat=[]
    for i in range(n):
        row=[]
        for j in range(n):
            x=rd.random() #this is in [0,1)
            y=N(x) #I can transform N(x) if I want
            row.append(y)
        mat.append(row)
    return mat

In [82]:
def flip_I(k,n):
    #generates unitary nxn matrix that flips the sign of k-th eigenvalue
    mat=np.identity(n)
    mat[k,k]=-1
    return mat

Part 2. Function that generates an orthonormal matrix

In [83]:
def ortho_mat(n):
    #uses an array of symmetric matrices to get a nxn orthogonal matrix
    A=g_mat(n)
    Q=Householder(A)[0]
    return Q

Part 3. Functions that generate Hessenberg and Tridigonal symmetric matrices

In [84]:
def Hessenberg (B):
    #turns an nxn random matrix into Hessenberg form
    cpy=B.copy()
    b=extract_minor(cpy)
    H=Householder(b)[1].tolist() #we turn the lower n-1xn-1 minor of A into a triangular matrix
        
    for i in range(len(cpy)-1):#then we put the hessenberg matrix together
        H[i].append(cpy[i+1][len(cpy)-1])
    H.insert(0, cpy[0])
    
    return H

In [85]:
def Trid (B):
    #turns a given random nxn matrix into a symmetric tridiagonal matrix
    H=Hessenberg (B)
    trid=H.copy()
    for i in range(len(B)):
        for j in range(len(B)):
            if abs(i-j)>1:
                trid[i][j]=0
            else :
                if i>j:
                    trid[i][j]=trid[j][i]
                else:
                    m=0
    return trid

Part 5. function that generates a random symmetric matrix wiht prescribed eigenvalues

In [86]:
def symm_mat(L):
    #given a set of n eigenvalues, generates a random symmetric nxn matrix with same set of eigenvalues
    Q=ortho_mat(len(L))
    A=np.matmul(np.transpose(Q), np.matmul(L,Q))
    return A

Norm functions :

In [87]:
def L2_norm(vector):
    n=len(vector)
    norm=0
    for i in range(n):
        temp=vector[i]*vector[i]
        norm+=temp
    norm=math.sqrt(norm)
    return norm

In [88]:
def matrix_norm(mat):
    #computes the L_{\infty} norm of an nxn matrix
    n=len(mat)
    norm=0
    for i in range(n):
        for j in range(n):
            temp=abs(mat[i][j])
            if temp>norm:
                norm=temp
    return norm

Matrix manipulation functions :

In [89]:
def extract_minor(B):
    #outputs n-1 minor of A an nxn matrix
    mat=[]
    for i in range(len(B)-1):
        row=[]
        for j in range(len(B)-1):
            row.append(B[i+1][j+1])
        mat.append(row)
    return mat

In [90]:
def sizedQ (R, k):
    #takes a submatrix and reformats it into kxk
    I=np.identity(k-len(R))
    mat= np.block([
    [I,               np.zeros((k-len(R), len(R)))],
    [np.zeros((len(R), k-len(R))), R               ]
    ])
    return mat

Part 1. QR factorisation using Householder reflections :

In [91]:
def find_Q_list(B):
    #part of Householder reflections
    Q_list=[]
    n=len(B)

    for i in range(n-1):
        x=[]
        for j in range(len(B)):
            x.append(B[j][0])#extract first row
        alpha=L2_norm(x)
        u=x-alpha*np.identity(n-i)[0]
        v=u/L2_norm(u)
        Q=np.identity(n-i)-2*np.outer(np.transpose(v),v)
        Q_sized=sizedQ(Q,n)
        Q_list.append(Q_sized)
        B= np.matmul(Q, B)
        B= extract_minor(B)
    return Q_list

In [92]:
def find_Q_R(Q_list, B):
    #part of Householder reflections
    R=Q_list[len(Q_list)-1]
    Q=np.transpose(Q_list[0])

    for i in range(len(Q_list)-1):
        R=np.matmul(R, Q_list[len(Q_list)-2-i])
        Q=np.matmul(Q, np.transpose(Q_list[i+1]))
    
    R=np.matmul(R,B)
    return [Q, R]

In [93]:
def post_prod(Q,R):
    #multiplies by adequate matrix to ensure T has nonneg diagonals
    test=Q.copy()
    temp=R.copy()
    for i in range(len(temp)):
        if temp[i][i]<0:
            temp=np.matmul(flip_I(i, len(temp)), temp)
            test=np.matmul(test, np.transpose(flip_I(i,len(temp))))
    return [test, temp]

In [94]:
def Householder (B):
    #performs QR diagonalisation using HH relfections 
    #returns Q orthonormal and R triangular st QR=A and R has nnneg entries
    B=np.array(B)
    og=B.copy()
    
    Q_list=find_Q_list(B)
    temp=find_Q_R(Q_list, og)
    final=post_prod(temp[0], temp[1])
    
    return final

Part 4. QR factorisaiton using Rayleigh Shifts :

In [103]:
def deflate(mat):
    #boolean function that checks if the tridiagonal matrix is ready for deflation
    #or if it's ready for recording
    boolean=1
    index=-1
    record=0
    if len(mat)>1:
        for i in range(len(mat)-1):
            if abs(mat[i][i+1]) < eps:
                if abs(mat[i+1][i]) <eps :
                    boolean=0
                    index=i
                else:
                    p=0
            else:
                q=0
    else:
        boolean=0
        record=1
    return [boolean, index, record]

In [104]:
def split(index, B):
    #given a pivot, splits a square matrix along the diagonal into 2 square submatrices
    #print ("This is the matrix Im tryna split : ", B)
    #print ("The pivot is at ", index)
    temp=[]
    if index > -1 :
        temp1=[]
        for i in range(index+1):
            row=[]
            for j in range(index+1):
                row.append(B[i][j])
            temp1.append(row)
        temp.append(temp1)
        temp2=[]
        for i in range(len(B)-index-1):
            row=[]
            for j in range(len(B)-index-1):
                row.append(B[i+index+1][j+index+1])
            temp2.append(row)
        temp.append(temp2)
    return temp

In [105]:
def find_nu_list(B, boolean):
    #computes the eigenvalues of a given tridiagonal nxn matrix
    og=B.copy()
    nu_list=[]
    B_list=[og]
    i=0
    while len(nu_list)<len(og): #while we havent found all eigenvalues yet
        #print ("Am I stuck in a while loop?")
        M=B_list[i]
        while deflate(M)[0]:#while the off diagonal entries aren't small enough yet
            #print ("Am I stuck in a while loop?")
            if(boolean):
                nu= RQ(M) #do RS!
            else:
                nu= WS(M) #do WS!
            mat=M-nu*np.identity(len(M))
            QR=Householder(mat)
            M=np.matmul(QR[1], QR[0])+nu*np.identity(len(M))
        B_list=B_list+split(deflate(M)[1], M) #now work on minors
        if deflate(M)[2]:#if the minors are unitary
            nu_list.append(M[0][0]) #we've found an eigenvalue!
        i+=1    
    return nu_list

In [106]:
def RQ(B):
    #computes dominating eigenvalue of A using Raleigh quotient
    nu=20.
    b=np.ones((len(B),1))

    for i in range(10):
        mat=np.linalg.inv(B-nu*np.identity(len(B)))
        temp=np.matmul(mat, b)
        alpha=L2_norm(temp)
        b=temp/alpha
        b_t=np.transpose(b)
    
        num=np.matmul(b_t, np.matmul(B,b))
        den=np.matmul(b_t,b)
        nu=num/den
    
    return nu[0][0]

In [107]:
def Rayleigh(B):
    #performs Qr decomposition on a nxn tridiagonal matrix
    eigenvalues=find_nu_list(B,1)
    R=[[0. for i in range(len(B))] for j in range(len(B))]
    for k in range(len(B)):
        R[k][k]=eigenvalues[k]
    Q=np.matmul(B, np.linalg.inv(R))
    final=post_prod(Q,R)
    return final
#note that there's a bug here right now, Q isn't orthonormal, it's just a random matrix. I'm emailing about this       

Part 4. QR factorisation using Wilkinson shifts :

In [108]:
def WS_eig(b):
    #computes theeigenvalue of a 2x2submat using Wilkinson shifts
    sgn=1
    if len(b)==2:
        delta=(b[0][0]-b[1][1])/2
        root=math.sqrt(delta*delta+b[0][1]*b[0][1])
        if abs(delta)>0:
            sgn=abs(delta)/delta
        nu=b[1][1]-sgn*b[0][1]/(abs(delta)+root)
    else:
        nu=b[0][0]
    return nu

In [109]:
def WS(B):
    #given a nxn tridiagonal matrix, computes Wilkinson shift
    b=[]
    for i in range(2):
        row=[]
        for j in range(2):
            row.append(B[len(B)-2+i][len(B)-2+j])
        b.append(row)
    nu=WS_eig(b)
    return nu

In [110]:
def Wilkinson(B):
    #performs Qr decomposition on a nxn tridiagonal matrix
    eigenvalues=find_nu_list(B,0)
    R=[[0. for i in range(len(B))] for j in range(len(B))]
    for k in range(len(B)):
        R[k][k]=eigenvalues[k]
    Q=np.matmul(B, np.linalg.inv(R))
    final=post_prod(Q,R)
    return final
#note that there's a bug here right now, Q isn't orthonormal, it's just a random matrix. I'm emailing about this 

In [78]:
mu=0.
sigma=1.
rd.seed(1)
eps=0.000001
n=6

L=[(i+1)*np.identity(n)[i] for i in range(n)]
print (L)
print (symm_mat(L))






[array([1., 0., 0., 0., 0., 0.]), array([0., 2., 0., 0., 0., 0.]), array([0., 0., 3., 0., 0., 0.]), array([0., 0., 0., 4., 0., 0.]), array([0., 0., 0., 0., 5., 0.]), array([0., 0., 0., 0., 0., 6.])]
[[ 3.42937224  0.51844959 -0.60251043  0.80063775  0.66699488 -1.09067671]
 [ 0.51844959  3.19261447  0.16482415  0.25105202  0.61132038  0.07567567]
 [-0.60251043  0.16482415  4.9509587  -0.61772163  0.20649638 -0.74118231]
 [ 0.80063775  0.25105202 -0.61772163  4.8485183  -0.28870584 -0.72561521]
 [ 0.66699488  0.61132038  0.20649638 -0.28870584  2.65137883 -0.4254578 ]
 [-1.09067671  0.07567567 -0.74118231 -0.72561521 -0.4254578   1.92715747]]
